In [1]:
import pandas as pd
from itertools import combinations
import yaml
import string

# User-defined settings
yaml_path = 'C:/Users/zachh/Downloads/00 Pokemon Code/base_files/COMPLETEDEX.yaml'  # Path to the YAML file
use_fairy_types = 'yes'  # 'yes' for type1 and type2, 'no' for nf_type1 and nf_type2
soullink_csv_path = 'C:/Users/zachh/Downloads/00 Pokemon Code/base_files/soullink_mons.csv' # Path to the "soullink mons" CSV file
min_team_size = 5  # Minimum number of team members

# Load Pokémon data from the YAML file
def load_pokemon_data(yaml_path, use_fairy_types):
    type_suffix = '' if use_fairy_types.lower() == 'yes' else 'nf_'
    type1_col = f"{type_suffix}type1"
    type2_col = f"{type_suffix}type2"
    with open(yaml_path, 'r', encoding='utf-8') as file:
        data = yaml.safe_load(file)
        pokemon_df = pd.DataFrame.from_dict(data, orient='index')
        pokemon_df['type1'] = pokemon_df[type1_col]
        pokemon_df['type2'] = pokemon_df[type2_col].replace('Monotype', None)  # Handle 'Monotype' as None
        pokemon_df['bst'] = pokemon_df['bst'].astype(int)  # Ensure BST is an integer
        return pokemon_df[['bst', 'type1', 'type2']].rename_axis('name').reset_index()

# Load the "soullink mons" CSV file
soullink_df = pd.read_csv(soullink_csv_path)

# Function to clean and normalize names
def clean_and_normalize(name):
    name = name.lower()
    name = name.translate(str.maketrans("", "", string.punctuation))
    return name

# Clean and normalize names in "soullink mons" dataframe
soullink_df['player1'] = soullink_df['player1'].apply(clean_and_normalize)
soullink_df['player2'] = soullink_df['player2'].apply(clean_and_normalize)

# Load Pokémon data with preferred type
pokemon_df = load_pokemon_data(yaml_path, use_fairy_types)

# Normalize names in Pokémon DataFrame
pokemon_df['name'] = pokemon_df['name'].map(clean_and_normalize)

# Merge dataframes based on the Pokémon names
merged_df = pd.merge(soullink_df, pokemon_df, left_on='player1', right_on='name', suffixes=('', '_1'))
merged_df = pd.merge(merged_df, pokemon_df, left_on='player2', right_on='name', suffixes=('_1', '_2'))

# Ensure that you are dropping the correct columns
cols_to_drop = [col for col in ['name_1', 'name_2'] if col in merged_df.columns]
merged_df = merged_df.drop(columns=cols_to_drop)

# Validate combinations based on types
def is_valid_combination(combination):
    all_types = [t for pair in combination for t in (pair['type1_1'], pair['type2_1'], pair['type1_2'], pair['type2_2']) if t]
    return len(all_types) == len(set(all_types))

# Calculate the sum of BST values in a combination
def calculate_bst_sum(combination):
    return sum(pair['bst_1'] + pair['bst_2'] for pair in combination)

# Calculate individual BST sums
def calculate_individual_bst_sums(combination):
    bst_sum_player1 = sum(pair['bst_1'] for pair in combination)
    bst_sum_player2 = sum(pair['bst_2'] for pair in combination)
    return bst_sum_player1, bst_sum_player2

# Generate all possible combinations
all_combinations = []
for r in range(min_team_size, 8):  # 8 because the range function is exclusive on the upper bound
    pairs_combinations = combinations(merged_df.to_dict('records'), r)
    valid_combinations = list(filter(is_valid_combination, pairs_combinations))
    all_combinations.extend(valid_combinations)

# Sort combinations by the sum of BST values
all_combinations.sort(key=calculate_bst_sum, reverse=True)

# Print results
for idx, combination in enumerate(all_combinations, 1):
    print(f"Combination {idx} (Party Size: {len(combination)}):")
    if not combination:
        print("No valid combinations.")
    else:
        for i, pair in enumerate(combination, 1):
            name1, name2 = pair['player1'].capitalize(), pair['player2'].capitalize()
            types = [t for t in (pair['type1_1'], pair['type2_1'], pair['type1_2'], pair['type2_2']) if t]
            types_str = ', '.join(types)
            print(f"{i}: {name1} & {name2} (Types: {types_str})")
        bst_sum = calculate_bst_sum(combination)
        bst_sum_player1, bst_sum_player2 = calculate_individual_bst_sums(combination)
        print(f"Total BST Sum: {bst_sum}")
        print(f"Individual BST Sums - Player 1: {bst_sum_player1}, Player 2: {bst_sum_player2}")
        print("---")
